1. 문서 내용 읽기
2. 문서 분할(쪼개기)
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 -> 답변 늦어짐
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있으면, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
## 패키지/라이브러리 설치
# %pip install -Uq docx2txt langchain_community langchain-text-splitters langchain_chroma


Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] <archive url/path> ...

no such option: --upgrage


In [ ]:
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1. 문서 내용 읽고, 분할 ===========================================================
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1500,
    chunk_overlap= 200,
    length_function= len,
    is_separator_regex= False,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장 =============================================
## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
## 2.3 벡터 데이터베이스에 저장
## [방법 1] in memory
# database = Chroma.from_documents(
#     documents= document_list,
#     embedding=embedding,
# )

In [2]:
%pip install langchain-pinecone

  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.12.6
    Uninstalling aiohttp-3.12.6:
      Successfully uninstalled aiohttp-3.12.6
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

In [8]:
from langchain_pinecone import PineconeVectorStore

database = PineconeVectorStore.from_documents(
    documents= document_list,
    embedding=embedding,
    index_name='law-index',
)

database

In [ ]:
# 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ================================

## 3.1 사용자 질문
query = '전세사기 피해자 금융지원에 대해 설명해주세요.'

## 3.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

# 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ============================
## 4.1 프롬프트 작성
prompt = '''
[identity]
-당신은 전세사기 피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''
## 4.2 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)

## 4.3 LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model = 'gpt-4o',)

## 4.4 LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기 피해자 금융지원에 관련된 내용은 특별법에서 다음과 같이 규정되어 있습니다:\n\n1. **융자 및 금융지원 제공**: 국가 및 지방자치단체는 전세사기 피해자 및 특정 임차인에게 긴급한 주거안정을 위해 필요한 자금을 융자하거나 그 외의 금융지원을 제공할 수 있습니다.\n\n2. **주택도시기금 지원**: 피해자가 주택임대차보호법에 따른 우선변제를 받지 못하여 시급한 지원이 필요하다고 판단될 경우, 주택도시기금에서 해당 임대차에 필요한 자금을 융자할 수 있습니다.\n\n3. **신용정보 유예 또는 삭제**: 한국자산관리공사와 같은 금융회사들이 전세사기 피해자와 관련된 대출에 대해 채무 불이행 및 대위변제의 등록을 유예하거나 삭제할 수 있습니다. 이는 전세사기 피해자들이 신용 문제로부터 보호받도록 하는 조치입니다.\n\n4. **주택저당채권대상 포함**: 전세사기 피해자 및 관련된 임차인에 대해서는 주택저당채권 대상에 오피스텔과 같은 준주택을 포함시켜, 이들 주거 목적으로 사용할 수 있도록 지원합니다.\n\n이와 같은 지원 조치는 피해자가 전세사기로 인해 주거 안정을 해치지 않도록 돕고, 경제적 어려움을 완화하기 위한 것입니다. 이러한 규정들은 피해자들이 안정적으로 주거 환경을 유지할 수 있게 하는 데 중점을 두고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 349, 'prompt_tokens': 4168, 'total_tokens': 4517, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tok